In [88]:
import math  # 数学関数（sqrt, log など）
from symbol import continue_stmt

import numpy as np  # 乱数や配列の補助（主に初期化用）
import torch  # テンソル演算の基盤
import matplotlib.pyplot as plt
from botorch.models import SingleTaskGP  # 単一タスクのガウス過程モデル
from botorch import fit_gpytorch_model  # GPyTorch モデルの学習ユーティリティ
from botorch.acquisition import ExpectedImprovement  # 取得関数 EI
from gpytorch.mlls import ExactMarginalLogLikelihood  # 厳密周辺尤度（GP 学習用）
from gpytorch.kernels import RBFKernel, ScaleKernel  # RBF(ARD) + スケールカーネル
from gpytorch.constraints import GreaterThan  # （必要なら）ハイパーパラメータ下限
import warnings  # 警告抑制
warnings.filterwarnings("ignore")  # 学習時の警告を見やすさのため抑制

torch.set_default_dtype(torch.double)  # BoTorch/GPyTorch は double が安定

In [89]:
d = 2
bounds = torch.tensor([[-5.0] * d, [5.0] * d])

In [90]:
bounds

tensor([[-5., -5.],
        [ 5.,  5.]])

In [91]:
lb,ub = bounds[0] , bounds[1]

In [92]:
lb

tensor([-5., -5.])

In [93]:
X = torch.rand(20, d, dtype=bounds.dtype) * (ub - lb) + lb
X

tensor([[-3.3882,  1.0072],
        [-0.2345, -0.6464],
        [ 4.9067, -3.1632],
        [ 1.5658,  3.0287],
        [ 3.8700, -3.5401],
        [-0.4563, -2.9145],
        [-1.3361, -2.3061],
        [ 0.6140,  3.1755],
        [-0.7486,  2.7690],
        [-2.5182, -1.2593],
        [-1.2513,  3.9609],
        [-4.0791,  1.4844],
        [ 3.6436, -0.8377],
        [ 4.1174,  3.8554],
        [ 2.8339,  2.1739],
        [ 3.5496,  2.7040],
        [-1.1774, -4.8876],
        [-4.5311,  2.6751],
        [ 1.4907, -2.4998],
        [-4.3968,  0.8139]])

In [94]:
def styblinski_tang(x: torch.Tensor) -> torch.Tensor:  # ベンチ関数（ベクトル化対応）
    z = x  # そのまま使う（前処理なし）
    return 0.5 * torch.sum(z**4 - 16.0 * z**2 + 5.0 * z, dim=-1)  # 定義式

In [95]:
styblinski_tang(X[1])

tensor(-5.8969)

In [96]:
styblinski_tang(X[2])

tensor(71.5864)

In [97]:
with torch.no_grad():
    y = styblinski_tang(X)

In [98]:
y

tensor([-39.5000,  -5.8969,  71.5864, -36.4340, -28.5646, -41.9489, -50.1974,
        -23.2998, -31.2205, -51.4973,  -6.9686, -16.3730, -16.4334,  19.5646,
        -46.1204, -37.5506,  68.9386,  10.2221, -48.2982,  18.1687])

In [99]:
Y = y.reshape(-1, 1).double()
Y

tensor([[-39.5000],
        [ -5.8969],
        [ 71.5864],
        [-36.4340],
        [-28.5646],
        [-41.9489],
        [-50.1974],
        [-23.2998],
        [-31.2205],
        [-51.4973],
        [ -6.9686],
        [-16.3730],
        [-16.4334],
        [ 19.5646],
        [-46.1204],
        [-37.5506],
        [ 68.9386],
        [ 10.2221],
        [-48.2982],
        [ 18.1687]])

In [100]:
def _fit_gp():
    base = RBFKernel(ard_num_dims=d, lengthscale_constraint=GreaterThan(0.0) if 0.0 > 0 else None)
    kernel = ScaleKernel(base).to(X)
    model = SingleTaskGP(X, Y, covar_module=kernel)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)

In [101]:
_fit_gp()

In [102]:
idx = int(torch.argmin(Y).item())
idx

9

In [103]:
best_x = X[idx].detach().clone()
print(best_x)
best_y = float(Y[idx].item())
print(best_y)

tensor([-2.5182, -1.2593])
-51.497252194929416


In [104]:
n_eval = X.shape[0]
n_eval

20

In [105]:
total_iteration = 0
total_iteration += 1

In [106]:
A = torch.eye(d, dtype=bounds.dtype)
A

tensor([[1., 0.],
        [0., 1.]])

In [107]:
b = torch.zeros(d, dtype=bounds.dtype)
b

tensor([0., 0.])

In [108]:
total_iteration

1

In [109]:
def _beta_t() -> float:
    sigma = 1.0
    lam = 1.0
    delta = 0.1
    S = 1.0
    t = max(1, total_iteration)
    val = sigma * math.sqrt(d * math.log(max(1e-9, 1 + (t - 1) / lam) / delta)) + math.sqrt(lam) * S
    return float(val)

In [110]:
_beta_t()

3.145966026289347

In [111]:
torch.inverse(A)

tensor([[1., 0.],
        [0., 1.]])

In [112]:
theta_hat = torch.inverse(A) @ b
theta_hat

tensor([0., 0.])

In [113]:
def _select_direction_continuous_fixed() -> torch.Tensor:
    global A
    print(A)
    A = 0.5 * (A + A.t())  # 数値安定のため対称化
    A_inv = torch.inverse(A)  # A の逆行列（小規模 d を想定）
    theta_hat = A_inv @ b  # θ̂ = A^{-1} b（LinUCB の重み推定）
    beta = _beta_t()  # β_t（信頼幅）
    x = theta_hat.clone()  # 初期ベクトルを θ̂ から開始
    if float(x.norm()) < 1e-12:  # ほぼゼロなら乱数で初期化
        x = torch.randn_like(theta_hat)  # ランダム初期化
    x = x / (x.norm() + 1e-12)  # 単位ベクトル化
    for _ in range(50):  # 固定点反復（上限50回）
        y = A_inv @ x  # y = A^{-1} x
        denom = torch.sqrt(torch.clamp(x @ y, min=1e-18))  # sqrt(x^T A^{-1} x)
        z = theta_hat + beta * (y / denom)  # z = θ̂ + β * A^{-1}x / sqrt(x^T A^{-1} x)
        x_new = z / (z.norm() + 1e-12)  # 正規化
        if float((x_new - x).norm()) < 1e-8:  # 収束チェック
            x = x_new  # 収束
            break  # 反復終了
        x = x_new  # 継続
    return x  # 連続最適方向（近似）


In [114]:
direction = _select_direction_continuous_fixed()
direction

tensor([[1., 0.],
        [0., 1.]])


tensor([0.8333, 0.5528])

In [115]:
dir_unit = direction / (direction.norm() + 1e-12 )
dir_unit

tensor([0.8333, 0.5528])

In [116]:
lb, ub = bounds[0], bounds[1]
lb

tensor([-5., -5.])

In [117]:
t_low, t_high = -float("inf"), float("inf")
t_low

-inf

In [118]:
a = float(dir_unit[0].item())
a

0.8332922337721866

In [120]:
if abs(a) < 1e-12:
    print("小さすぎ！")

In [123]:
lo = (float(lb[0].item()) - float(best_x[0].item())) / a
hi = (float(ub[0].item()) - float(best_x[0].item())) / a
lo

-2.9782605568231237

In [124]:
a.view(-1, 1)

AttributeError: 'float' object has no attribute 'view'